# Prediction with .h5 file

### Imports


In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import numpy as np

import tools

# Root CSV files directory
dirname = "./prediction_data/"

# Frame count
frames = 100

In [2]:
model = tf.keras.models.load_model('sign_lang_recognition.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 32)           12416     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 59,557
Trainable params: 59,557
Non-trainable params: 0
_________________________________________________________________


#### Load the file to predict

In [3]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train=np.array(features)

tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([labels])[0]
y_train = to_categorical(encoded_train)
print(y_train)

{'deutschland': 1}
[[0. 1.]
 [0. 1.]]


In [4]:
print(x_train)

[[[0.507547 0.382073 0.213684 ... 0.       0.       0.      ]
  [0.50686  0.383572 0.274671 ... 0.834884 0.737334 0.852904]
  [0.505028 0.391949 0.284391 ... 0.751541 0.714473 0.775634]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.505633 0.400101 0.451716 ... 0.       0.       0.      ]
  [0.502219 0.403942 0.451105 ... 0.       0.       0.      ]
  [0.505479 0.400694 0.448652 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]]


In [5]:
y_pred = model.predict(x_train)

In [6]:
predictions = np.array([np.argmax(pred) for pred in y_pred])
print(predictions)

[1 2]


https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python

In [38]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
# Test model on random input data.

#interpreter.set_tensor(0.0, x_train)

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

print(input_data)
interpreter.set_tensor(input_details[0]['index'], input_data)

#interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
#output_data = interpreter.get_tensor(output_details[0]['index'])
#print(output_data)

[{'name': 'lstm_input', 'index': 0, 'shape': array([  1, 100,  86]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[[[0.29059538 0.40627855 0.53578615 ... 0.00327641 0.0639431  0.40411133]
  [0.8692346  0.08685412 0.69634986 ... 0.9974383  0.38653663 0.6258291 ]
  [0.8929269  0.11467202 0.9103828  ... 0.12192424 0.05083548 0.5769115 ]
  ...
  [0.25443268 0.9936906  0.8852658  ... 0.6043434  0.3073329  0.6151898 ]
  [0.5280396  0.98501647 0.4209363  ... 0.66375786 0.77141243 0.8421129 ]
  [0.21817924 0.2039082  0.7296217  ... 0.3307883  0.41537222 0.27234465]]]


In [39]:
print(input_data)
interpreter.invoke()

[[[0.29059538 0.40627855 0.53578615 ... 0.00327641 0.0639431  0.40411133]
  [0.8692346  0.08685412 0.69634986 ... 0.9974383  0.38653663 0.6258291 ]
  [0.8929269  0.11467202 0.9103828  ... 0.12192424 0.05083548 0.5769115 ]
  ...
  [0.25443268 0.9936906  0.8852658  ... 0.6043434  0.3073329  0.6151898 ]
  [0.5280396  0.98501647 0.4209363  ... 0.66375786 0.77141243 0.8421129 ]
  [0.21817924 0.2039082  0.7296217  ... 0.3307883  0.41537222 0.27234465]]]


In [40]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[1.5775078e-04 9.8049325e-01 3.0060988e-03 1.1607097e-03 1.5182310e-02]]
